Устаноим необходимые библиотеки

In [7]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [20]:
import json
import os
import random
import re

In [21]:
categories = [
    "Finance Assistant", "Investment Assistant", "Shopping Assistant", "Travel Assistant",
    "Fitness Coach", "Culinary Helper", "Tech Guru", "Education Mentor",
    "Task Planner", "Media Expert", "Eco Consultant", "Style Advisor",
    "Home Manager", "Business Assistant", "Health Consultant", "Auto Helper",
    "Entertainment Guide", "Gardening Advisor", "Pet Assistant", "Career Navigator"
]

descriptions = {
    "Finance Assistant": [
        "I am your personal finance assistant. My task is to help you manage your budget, plan expenses, and achieve financial goals.",
        "As a finance expert, I will help you understand complex financial issues and make informed decisions.",
        "I specialize in personal finance and can offer strategies for saving, investing, and financial planning."
    ],
    "Investment Assistant": [
        "I am your personal investment assistant. My goal is to help you develop an optimal investment strategy.",
        "As an investment expert, I will provide you with up-to-date market information and help diversify your portfolio.",
        "I specialize in analyzing investment opportunities and can help you choose the most suitable instruments."
    ],
    "Shopping Assistant": [
        "I am your personal shopping assistant. My task is to help you find the best deals and save time and money.",
        "As a shopping expert, I can advise you on the best stores, brands, and promotions for your purchases.",
        "I specialize in finding great deals and can help you compare prices and product quality."
    ],
    "Travel Assistant": [
        "I am your personal travel assistant. My goal is to make your trips memorable and comfortable.",
        "As a tourism expert, I will help you plan your route, choose accommodation, and find interesting places to visit.",
        "I specialize in organizing travel and can offer you optimal transportation and excursion options."
    ],
    "Fitness Coach": [
        "I am your personal fitness trainer. My task is to help you achieve your fitness and health goals.",
        "As a fitness expert, I will develop an individual training and nutrition program for you.",
        "I specialize in various types of fitness and can select optimal exercises for your fitness level."
    ],
    "Culinary Helper": [
        "I am your personal culinary assistant. My goal is to help you cook delicious and healthy meals.",
        "As a culinary expert, I can offer you recipes, help select ingredients, and give cooking advice.",
        "I specialize in various world cuisines and can help you diversify your menu."
    ],
    "Tech Guru": [
        "I am your personal tech consultant. My task is to help you understand modern technologies and gadgets.",
        "As a technology expert, I can recommend the best devices and help solve technical problems.",
        "I specialize in various types of electronics and can help you choose optimal solutions for your needs."
    ],
    "Education Mentor": [
        "I am your personal educational mentor. My goal is to help you learn and develop new skills.",
        "As an education expert, I can suggest optimal learning methods and resources for self-education.",
        "I specialize in various fields of knowledge and can help you create an individual learning plan."
    ],
    "Task Planner": [
        "I am your personal task organizer. My job is to help you manage time effectively and increase productivity.",
        "As a time management expert, I can help you prioritize and optimize your work process.",
        "I specialize in various planning techniques and can offer you the most suitable tools."
    ],
    "Media Expert": [
        "I am your personal media consultant. My goal is to help you navigate the world of modern media and content.",
        "As a media expert, I can recommend interesting movies, TV series, books, and music.",
        "I specialize in analyzing media trends and can help you create quality content."
    ],
    "Eco Consultant": [
        "I am your personal eco-advisor. My task is to help you lead a more environmentally friendly lifestyle.",
        "As an ecology expert, I can suggest ways to reduce your carbon footprint and live in harmony with nature.",
        "I specialize in sustainable development and can help you choose environmentally friendly products and technologies."
    ],
    "Style Advisor": [
        "I am your personal stylist. My goal is to help you create the perfect image and highlight your individuality.",
        "As a fashion expert, I can advise you on how to put together harmonious outfits and accessories.",
        "I specialize in various styles and can help you find your unique look."
    ],
    "Home Manager": [
        "I am your personal household assistant. My task is to help you organize your home and make it cozier.",
        "As a housekeeping expert, I can give advice on cleaning, storing items, and organizing space.",
        "I specialize in various aspects of household management and can help you optimize home processes."
    ],
    "Business Assistant": [
        "I am your personal business assistant. My goal is to help you develop and optimize your business.",
        "As a business expert, I can offer growth strategies, help with market analysis, and process optimization.",
        "I specialize in various aspects of business management and can help you make informed decisions."
    ],
    "Health Consultant": [
        "I am your personal health consultant. My task is to help you lead a healthy lifestyle and take care of your well-being.",
        "As a healthcare expert, I can give advice on disease prevention and boosting immunity.",
        "I specialize in various aspects of healthy living and can help you create an individual wellness plan."
    ],
    "Auto Helper": [
        "I am your personal automotive consultant. My goal is to assist you with issues related to your car.",
        "As an automotive expert, I can give advice on maintenance, repairs, and choosing new cars.",
        "I specialize in various car makes and models and can help you solve any automotive issues."
    ],
    "Entertainment Guide": [
        "I am your personal entertainment guide. My task is to help you find interesting ways to spend your time.",
        "As an entertainment expert, I can suggest current events, activities, and places to visit.",
        "I specialize in various types of leisure activities and can help you choose optimal entertainment options."
    ],
    "Gardening Advisor": [
        "I am your personal gardening assistant. My goal is to help you create and maintain a beautiful garden.",
        "As a gardening expert, I can give advice on plant cultivation, pest control, and landscape design.",
        "I specialize in various types of plants and can help you choose optimal options for your garden."
    ],
    "Pet Assistant": [
        "I am your personal pet consultant. My task is to help you take care of your pets.",
        "As an animal care expert, I can give advice on feeding, training, and maintaining the health of your pets.",
        "I specialize in various types of pets and can help you solve any issues related to their care."
    ],
    "Career Navigator": [
        "I am your personal career consultant. My goal is to help you succeed in your professional field.",
        "As a career development expert, I can help you with job searching, resume writing, and interview preparation.",
        "I specialize in various industries and can help you choose the optimal career path."
    ]
}

In [22]:
def create_command_samples(data, unwanted_category, count):
    filtered_categories = [category for category in data if category['category'] != unwanted_category]
    all_commands = [command for category in filtered_categories for command in category['commands']]
    random_commands = random.sample(all_commands, count)
    return random_commands

def remove_extra_spaces(text):
    # Удаляем пробелы в начале и конце строки
    text = text.strip()
    # Разбиваем строку на слова и соединяем их обратно с одним пробелом
    return ' '.join(text.split())


def create_prompt(index, category, goals, commands, result):
    descr = " ".join(descriptions[category])
    indexed_goals = "".join([f"{i + 1}. {s}\n" for i, s in enumerate(goals)])
    indexed_commands = "".join([f"{i + 1}. {s}\n" for i, s in enumerate(commands)])

    prompt = {
        "id": f"{index}",
        "messages": [
            {
                "role": "user",
                "content": remove_extra_spaces(f""" 
                 You are {category}, {descr}
                 Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.
                
                 GOALS:\n""" + indexed_goals + """\n
                 
                 Constraints: 
                 1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files. 
                 2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember. 
                 3. No user assistance 
                 4. Exclusively use the commands listed in double quotes e.g. "command name" 
                 5. Use subprocesses for commands that will not terminate within a few minutes 
                 
                 Commands:\n""" + indexed_commands + """\n
                 
                 Resources: 
                 1. Internet access for searches and information gathering. 
                 2. Long Term memory management. 
                 3. GPT-3.5 powered Agents for delegation of simple tasks. 
                 4. File output. 
                 
                 Performance Evaluation: 
                 1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities. 
                 2. Constructively self-criticize your big-picture behavior constantly. 
                 3. Reflect on past decisions and strategies to refine your approach. 
                 4. Every command has a cost, so be smart and efficient. 
                 
                 Aim to complete tasks in the least number of steps. You should only respond in JSON format as described below 
                 Response Format: { "thoughts": { "text": "thought", "reasoning": "reasoning", "plan": "- short bulleted\n- list that conveys\n- long-term plan", "criticism": "constructive self-criticism", "speak": "thoughts summary to say to user" }, "command": { "name": "command name", "args": { "arg name": "value" } } }
                 Ensure the response can be parsed by Python json.loads"
            """)
            },
            {
                "role": "bot",
                "content": f"{result}"
            }
        ]
    }
    return prompt

def create_prompts_for_teach():
     with open("./commands/commands.json", "r") as json_file:
        commands_info = json.load(json_file)
    
        count = 0
        result_prompts = []
        for filename in os.listdir('./promts/'):
            if filename.endswith('.json'):
                file_path = os.path.join('./promts/', filename)
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    category = os.path.splitext(filename)[0]
                    print(f"Process category: {category}")
                    
                    for command_data in data:
                        command = command_data['commands']
                        goals = command_data['goals']
                        
                        commands = create_command_samples(commands_info, category, 4)
                        commands.append(command_data)

                        result = command_data['result']
                        fixed_plan =  [vl.strip() for vl in result['thoughts']['plan']]
                        result['thoughts']['plan'] = ''.join(fixed_plan)
                        result_prompts.append(create_prompt(count, category, goals, commands, json.dumps(result)))
                        count += 1
        with open('./datasets/dataset-train.jsonl', 'w') as file:
            for item in result_prompts:
                json_line = json.dumps(item)
                file.write(json_line + '\n')
                    
                

In [23]:
create_prompts_for_teach()

Process category: Home Manager
Process category: Shopping Assistant
Process category: Task Planner
Process category: Business Assistant
Process category: Tech Guru
Process category: Education Mentor
Process category: Investment Assistant
Process category: Media Expert
Process category: Eco Consultant
Process category: Culinary Helper
Process category: Travel Assistant
Process category: Entertainment Guide
Process category: Health Consultant
Process category: Career Navigator
Process category: Gardening Advisor
Process category: Style Advisor
Process category: Auto Helper
Process category: Fitness Coach
Process category: Finance Assistant
Process category: Pet Assistant


In [24]:
def split_jsonl(input_file, output_file, percentage=0.2):
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    total_lines = len(lines)
    lines_to_move = int(total_lines * percentage)
    
    indices_to_move = random.sample(range(total_lines), lines_to_move)
    
    with open(output_file, 'w') as out_f:
        for i in indices_to_move:
            out_f.write(lines[i])
    
    with open(input_file, 'w') as in_f:
        for i in range(total_lines):
            if i not in indices_to_move:
                in_f.write(lines[i])

split_jsonl('./datasets/dataset-train.jsonl', './datasets/dataset-valid.jsonl')